In [1]:
import os, sys
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
from hist.intervals import clopper_pearson_interval
def get_yerr(num,den):
    return abs(clopper_pearson_interval(num.view(), den.view()) - num.view()/den.view())

# we suppress ROOT warnings where our input ROOT tree has duplicate branches - these are handled correctly.
import warnings
warnings.filterwarnings("ignore", message="Found duplicate branch ")
np.seterr(divide='ignore', invalid='ignore')

from hist.intervals import clopper_pearson_interval
def get_yerr(num,den):
    return abs(clopper_pearson_interval(num.view(), den.view()) - np.nan_to_num(num.view()/den.view(), False, 1))

In [2]:
! ls ../python/Nov11_2017/stacked_hists/ele_hists.pkl

../python/Nov11_2017/stacked_hists/ele_hists.pkl


In [3]:
# you can load the output!
import pickle
with open('../python/Nov11_2017/miso_hists/ele_hists.pkl', 'rb') as f:
    hists = pickle.load(f)

In [7]:
def compute_soverb(h, cut, ch, range_min = 0, range_max = 150, remove_ttH = True):
    
    data_by_ch = {
    'ele': 'SingleElectron',
    'mu': 'SingleMuon',
    'had': 'JetHT',
    }
    
    data_label = data_by_ch
    signal_by_ch = {
    'ele': ['ttHToNonbb_M125', 'GluGluHToWW_Pt-200ToInf_M-125', 'VH', 'VBFHToWWToLNuQQ_M-125_withDipoleRecoil'],
    'mu': ['ttHToNonbb_M125', 'GluGluHToWW_Pt-200ToInf_M-125', 'VH', 'VBFHToWWToLNuQQ_M-125_withDipoleRecoil'],
    }

    # get samples existing in histogram
    samples = [h.axes[0].value(i) for i in range(len(h.axes[0].edges))]
    signal_labels = [label for label in samples if label in signal_by_ch[ch]]
    bkg_labels = [label for label in samples if (label and label != data_label and label not in signal_labels)]
    
    if remove_ttH:
        signal_labels.remove("ttHToNonbb_M125")

    # data
    data = None
    if data_label in h.axes[0]:
        data = h[{"samples": data_label}]

    # signal
    signal = [h[{"samples": label}] for label in signal_labels]
    # sum all of the signal
    if len(signal) > 0:
        tot_signal = None
        for i, sig in enumerate(signal):
            if tot_signal == None:
                tot_signal = signal[i].copy()
            else:
                tot_signal = tot_signal + signal[i]

    totsignal_val = tot_signal.values()

    # background
    bkg = [h[{"samples": label}] for label in bkg_labels]
    # sum all of the background
    if len(bkg) > 0:
        tot = bkg[0].copy()
        for i, b in enumerate(bkg):
            if i > 0:
                tot = tot + b

        tot_val = tot.values()
        tot_val_zero_mask = (tot_val == 0)
        tot_val[tot_val_zero_mask] = 1

    # replace values where bkg is 0
    totsignal_val[tot_val==0] = 0

    # integrate soverb in a given range for lep_fj_m
    bin_array = tot_signal.axes[0].edges[:-1] # remove last element since bins have one extra element
    condition = (bin_array>=range_min) & (bin_array<=range_max)

    s = totsignal_val[condition].sum()
    b = np.sqrt(tot_val[condition].sum())

    soverb_integrated = round((s/b).item(),2)
    print(f"Cut lep_miso<{cut}: S/sqrt(B) in range [{range_min}, {range_max}] of (Jet-Lep).mass is: {soverb_integrated}")
   
    return soverb_integrated

In [8]:
for cut in hists.keys():
    compute_soverb(hists[cut], cut, 'ele', range_min = 0, range_max = 150, remove_ttH = True)

Cut lep_miso<(1e-05, 1e-05): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.16
Cut lep_miso<(1e-05, 5e-05): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.16
Cut lep_miso<(1e-05, 0.0001): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.16
Cut lep_miso<(1e-05, 0.0005): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.16
Cut lep_miso<(1e-05, 0.001): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.16
Cut lep_miso<(1e-05, 0.005): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.17
Cut lep_miso<(1e-05, 0.01): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.17
Cut lep_miso<(1e-05, 0.05): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.17
Cut lep_miso<(1e-05, 0.1): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.17
Cut lep_miso<(5e-05, 1e-05): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.16
Cut lep_miso<(5e-05, 5e-05): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is: 0.16
Cut lep_miso<(5e-05, 0.0001): S/sqrt(B) in range [0, 150] of (Jet-Lep).mass is